In [88]:
import pandas as pd

sheet_id = "1kBPSqge0PG6TlHsk46MDq1XD-s2UNeceRxEGqGOTOgo"
sheet_name = 'Sheet1'

url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df = pd.read_csv(url)
df = df[['month', 'year', 'component', 'question number', 'question', 'answers', 'topic']]


In [89]:
df

,month,year,component,question number,question,answers,topic
0,March,2020,22,1,The formula of methane is CH4 and the formula ...,description of diffusion relative rate of diff...,Organic chemistry
1,March,2020,22,2,Which test is used to show that a sample of wa...,Evaporate the water to see if any solids remai...,Experimental techniques
2,March,2020,22,3,Chromatography is used to separate and identif...,colouring agent B display agent C finding agen...,Experimental techniques
3,March,2020,22,4,Lithium reacts with fluorine to form the compo...,Each fluorine atom gains one electron. B Each ...,"Atoms,elements and compounds"
4,March,2020,22,5,14\n6 and C12\n6 are isotopes of carbon.\nWhic...,12 6 is more reactive than C14 6 because the a...,"Atoms,elements and compounds"
...,...,...,...,...,...,...,...
800,November,2021,23,36,NaN,NaN,Organic chemistry
801,November,2021,23,37,NaN,NaN,Organic chemistry
802,November,2021,23,38,NaN,NaN,Organic chemistry
803,November,2021,23,39,NaN,NaN,Organic chemistry


In [90]:
df[['year', 'component','question number']] = df[['year', 'component', 'question number']].astype(int)

In [91]:
month_map = {'march': 'm', 'june': 's', 'november': 'w'}

def convert_month(month):
    return month_map[month.strip().lower()]

df['month'] = df['month'].apply(convert_month)


In [92]:
df

,month,year,component,question number,question,answers,topic
0,m,2020,22,1,The formula of methane is CH4 and the formula ...,description of diffusion relative rate of diff...,Organic chemistry
1,m,2020,22,2,Which test is used to show that a sample of wa...,Evaporate the water to see if any solids remai...,Experimental techniques
2,m,2020,22,3,Chromatography is used to separate and identif...,colouring agent B display agent C finding agen...,Experimental techniques
3,m,2020,22,4,Lithium reacts with fluorine to form the compo...,Each fluorine atom gains one electron. B Each ...,"Atoms,elements and compounds"
4,m,2020,22,5,14\n6 and C12\n6 are isotopes of carbon.\nWhic...,12 6 is more reactive than C14 6 because the a...,"Atoms,elements and compounds"
...,...,...,...,...,...,...,...
800,w,2021,23,36,NaN,NaN,Organic chemistry
801,w,2021,23,37,NaN,NaN,Organic chemistry
802,w,2021,23,38,NaN,NaN,Organic chemistry
803,w,2021,23,39,NaN,NaN,Organic chemistry


In [93]:
import difflib
import numpy as np
real_topic_list = [
        "1 The particulate nature of matter",
        "2 Experimental techniques",
        "3 Atoms, elements and compounds",
        "4 Stoichiometry",
        "5 Electricity and chemistry",
        "6 Chemical energetics",
        "7 Chemical reactions",
        "8 Acids, bases and salts",
        "9 The Periodic Table",
        "10 Metals",
        "11 Air and water",
        "12 Sulfur",
        "13 Carbonates",
        "14 Organic chemistry",
      ]

def similar(seq1, seq2):
    return difflib.SequenceMatcher(a=seq1.lower(), b=seq2.lower()).ratio()

topics = []
for topic in df['topic']:
    if isinstance(topic, float):
        topics.append('')
        continue
    topic = topic.strip().lower()
    if topic.isdigit():
        topics.append(real_topic_list[int(topic)-1])
    else:
        similarity = []
        for real_topic in real_topic_list:
            num, name = real_topic.split(' ', 1)
            similarity.append(similar(name, topic))
        topics.append(real_topic_list[np.argmax(similarity)])

            
df['topic'] = topics


In [94]:
labeled = df[df.question.isna()].drop(columns=['question', 'answers'])

labeled = labeled[labeled['topic'] != '']
labeled['topic'].value_counts()

14 Organic chemistry                  87
3 Atoms, elements and compounds       74
7 Chemical reactions                  70
10 Metals                             65
8 Acids, bases and salts              46
11 Air and water                      40
6 Chemical energetics                 34
2 Experimental techniques             34
9 The Periodic Table                  33
4 Stoichiometry                       32
5 Electricity and chemistry           22
12 Sulfur                             20
1 The particulate nature of matter    12
13 Carbonates                         12
Name: topic, dtype: int64

In [95]:
texts = []
for i, row in labeled[['year', 'component','question number']].iterrows():
    select = df[(df['year'] == row['year']) & (df['component'] == row['component']) & (df['question number'] == row['question number'])]
    select = select.dropna()
    if len(select) == 0:
        texts.append(None)
    else:
        texts.append((select['question'] + ' ' + select['answers']).values[0])

In [96]:
labeled['text'] = texts

In [98]:
labeled.to_csv('component24_2020-2021_labeled.csv', index=False)